# Image Synthesis Module

Functions in this module provide a copy of a image that has been synthesized by one of these functions

In [77]:
# A set of functions that can apply different transformations on an existing image to synthesis a new image

from PIL import Image, ImageEnhance, ImageFilter
from skimage import io, data, transform, filters, color, util
import colorsys
import numpy
import math
import random

# Sets plugin for skimage, using PIL to keep read in image formats the same for arrays
io.use_plugin('pil')
image_path = "C:\\Users\\Graeme\\Desktop\\image5.jpg"

## IO Helper Functions

In [78]:
# Convert Skimage image to a PIL image
def skimage_to_pil(img):
    io.imsave('cpy.jpg', img)
    pil_img = Image.open('cpy.jpg')
    return pil_img

In [79]:
# Convert PIL image to a Skimage image
def pil_to_skimage(img):
    img.save('cpy.jpg', "JPEG")
    ski_img = io.imread('cpy.jpg', plugin='pil')
    return ski_img

## Image Synthesis Functions

In [22]:
# Change contrast, that eventually can return the negative at high enough values
def change_contrast(img, level):
    factor = (259 * (level + 255)) / (255 * (259 - level))
    def contrast(c):
        value = 128 + factor * (c - 128)
        return max(0, min(255, value))
    return img.point(contrast)

In [23]:
# Range is real numbers greater than 0 to infinity
def change_brightness(img, level):
    brightness = ImageEnhance.Brightness(img)
    return brightness.enhance(level)

In [24]:
# Flip image over the vertical axis
def flip_vertical(img):
    return img.transpose(Image.FLIP_LEFT_RIGHT)

In [25]:
# Flip image over the horizontal axis
def flip_horizontal(img):
    return img.transpose(Image.FLIP_TOP_BOTTOM)

In [26]:
# Flip image over both axis
def flip_diagonal(img):
    imgcpy = img.transpose(Image.FLIP_TOP_BOTTOM)
    return imgcpy.transpose(Image.FLIP_LEFT_RIGHT)

In [27]:
# By passing a new_size we have to consider that it may be under, which we technically don't want as we would just be
# creating copies of the image based on the code below
# Need to consider checking for size before this function if we choose to randomize the values, 
# or else we'll end up with alot of doubles
def pad_image(img, new_size):
    old_img = img
    old_size = old_img.size
    
    # Check that all dimensions are greater or equal so it doesn't crop
    if all(i >= j for i, j in zip(new_size, old_size)):
        new_img = Image.new("RGB", new_size)   ## luckily, this is already black!
        new_img.paste(old_img, (int((new_size[0]-old_size[0])/2), int((new_size[1]-old_size[1])/2)))
        return new_img
    else:
        return old_img

In [68]:
# Skew image using some math
def skew_image(img, angle):

    width, height = img.size
    
    # Get the width that is to be added to the image based on the angle of skew
    xshift = math.tan(abs(angle)) * height
    new_width = width + int((xshift))
    
    # Apply transform
    img = img.transform((new_width, height), Image.AFFINE,
            (1, angle, -xshift if angle > 0 else 0, 0, 1, 0), Image.BICUBIC)
    
    return img

In [30]:
# Seam carve image
def seam_carve_image(img):
    
    # Convert to skimage image
    img = pil_to_skimage(img)
    
    # Energy Map, used to determine which pixels will be removed
    eimg = filters.sobel(color.rgb2gray(img))
    
    # (Width, Height)
    img_Dimensions = img.shape
    
    # Squish width if width >= height, squish height if height > width
    if(img_Dimensions[0] >= img_Dimensions[1]):
        mode = 'vertical'
    else:
        mode = 'horizontal'
    
    # Number of seams to be removed, need to determine best way to randomize
    num_seams = 15
    
    # Number of pixels to keep along the outer edges
    border = 5
    
    img = transform.seam_carve(img, eimg, mode, num_seams)
    
    # Convert back to PIL image
    img = skimage_to_pil(img)
    
    return img

In [31]:
# Rotate image
def rotate(img, rotation_angle):
    try:
        img_rotated = img.rotate(rotation_angle)
        return img_rotated
    except IOError as e:
        print(e)

In [32]:
def scale(img, scaling_factor):
    try:
        original_width, original_height = img.size
        img.thumbnail((original_height*scaling_factor, original_width*scaling_factor), Image.ANTIALIAS)
        return img
    except IOError as e:
        print(e)

In [33]:
# Crop image
# TODO: this method still needs to be tweaked so that we dont kill the image (main obj is still visible)
def crop(img, scaling_factor_x, scaling_factor_y):
    try:
        original_width, original_height = img.size
        img = img.crop((0, 0, int(original_width*scaling_factor_x), int(original_height*scaling_factor_y)))
        return img
    except Exception as e:
        print(e)

In [62]:
# Apply white noise to image
def white_noise(img):
    # Convert to skimage image
    img = pil_to_skimage(img)
        
    img = util.img_as_ubyte(util.random_noise(img, mode='s&p', seed=None, clip=True))
    img = util.img_as_ubyte(util.random_noise(img, mode='gaussian', seed=None, clip=True))
    img = util.img_as_ubyte(util.random_noise(img, mode='speckle', seed=None, clip=True))
    
    # Convert to PIL image
    img = skimage_to_pil(img)
    return img

In [35]:
def sharpen(img):
    img = img.filter(ImageFilter.SHARPEN)
    return img

In [36]:
# Apply a smooth filter to the image to smooth edges (blurs)
def soften(img):
    img = img.filter(ImageFilter.SMOOTH)
    return img

In [37]:
def grayscale(img):
    ''' grayscale
    im = Image.open(image_path) # open colour image
    im = im.convert('1') # convert image to black and white
    im.save(output, "JPEG")
    '''
        
    # black and white
    gray = img.convert('L')
    bw = gray.point(lambda x: 0 if x<128 else 255, '1')
    return bw

In [85]:
def hue_change(img):
    original_width, original_height = img.size
    print(str(original_width))
    print(str(original_height))
    ld = img.load()
    for y in range(original_height):
        for x in range(original_width):
            r,g,b = ld[x,y]
            h,s,v = colorsys.rgb_to_hsv(r/255., g/255., b/255.)
            h = (h + -90.0/360.0) % 1.0
            s = s**0.65
            r,g,b = colorsys.hsv_to_rgb(h, s, v)
            ld[x,y] = (int(r * 255.9999), int(g * 255.9999), int(b * 255.9999))
    return img

## Randomization and Synthesis Functions

In [88]:
def function_chooser(img, num):
    
    if num == 1:
        level = random.randint(0, 500)
        return change_contrast(img, level)
            
    elif num == 2: 
        level = random.randint(0, 500)
        return change_brightness(img, level)
            
    elif num == 3:
        return flip_vertical(img)
        
    elif num == 4:
        return flip_horizontal(img)
        
    elif num == 5:  
        return flip_diagonal(img)
        
    elif num == 6:
        width = random.randint(100, 1000)
        height = random.randint(100, 1000)
        new_size = (width, height)
        return pad_image(img, new_size)
        
    elif num == 7:
        degree = random.randint(-360,360)
        angle = math.radians(degree)
        return skew_image(img, angle)
            
    elif num == 8:
        return seam_carve_image(img)
        
    elif num == 9:
        rotation_angle = random.randint(0, 360)
        return rotate(img, rotation_angle)
        
    elif num == 10:
        scaling_factor = random.uniform(1, 5)
        return scale(img, scaling_factor)
    
    elif num == 11:
        scaling_factor_x = random.uniform(1, 5)
        scaling_factor_y = random.uniform(1, 5)
        return crop(img, scaling_factor_x, scaling_factor_y)
        
    elif num == 12:
        return white_noise(img)
        
    elif num == 13:
        return sharpen(img)
        
    elif num == 14:
        return soften(img)
        
    elif num == 15:
        return grayscale(img)
        
    elif num == 16:
        return hue_change(img)
        
    else:
        print("Didn't find a function that relates to that num...")
        return img   

In [96]:
img = Image.open(image_path)

def randomizor(img):
    
    # Randomize the number of synthesized images
    num_of_images = random.randint(1, 10)
    print("Number of images to synthesize: " + str(num_of_images))
    for count, images in enumerate(range(num_of_images), 1):
        
        print("Image: " + str(count))
        imgcpy = img
        # The number of transformations that will be applied
        num_of_operations = random.randint(1,5)
        for operations in range(0, num_of_operations):
            
            # The transformation function to be applied
            function_num = random.randint(1,16)
            print("Function applied: " + str(function_num))
            imgcpy = function_chooser(imgcpy, function_num)
        
        file_name = ".\\SynthesizedImages\\new_" + str(count) + ".jpg"
        imgcpy.save(file_name, "JPEG")
    
randomizor(img)

Number of images to synthesize: 8
Image: 1
Function applied: 5
Function applied: 16
153
240
Function applied: 2
Function applied: 10
Function applied: 1
Image: 2
Function applied: 7
Image: 3
Function applied: 8
Function applied: 3
Function applied: 16
138
240
Image: 4
Function applied: 2
Function applied: 9
Function applied: 1
Function applied: 2
Image: 5
Function applied: 15
Function applied: 8
Function applied: 11
Function applied: 9
Function applied: 10
Image: 6
Function applied: 15
Function applied: 11
Function applied: 14


C:\ProgramData\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


ValueError: image has wrong mode

In [97]:
### IGNORE FOR NOW

def main():
    img = change_contrast(image_path, 60)
    img = change_contrast_alt(image_path, 2)
    img = change_brightness(image_path, 2)
    img = flip_vertical(image_path)
    img = flip_horizontal(image_path)
    img = flip_diagonal(image_path)
    img = pad_image(image_path, (550, 600))
    img = skew_image(image_path, 0.3)
    
    img = rotate(image_path,45)
    img = scale(image_path, 0.1)
    img = crop(image_path, 0.5, 0.5)
    img = sharpen(image_path)
    img = soften(image_path)
    img = grayscale(image_path)
    img = hue_change(image_path)
  
if __name__== "__main__":
  main()

AttributeError: 'str' object has no attribute 'point'